In [1]:
from __future__ import absolute_import,division,print_function,unicode_literals
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import time
import re
import collections
import tensorflow_datasets as tfds
print (tf.__version__)

D:\Anaconda3.5.20\envs\tensorflow2.0\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda3.5.20\envs\tensorflow2.0\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda3.5.20\envs\tensorflow2.0\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda3.5.20\envs\tensorflow2.0\lib\site-packages\tensorflow\pyth

2.0.0-beta0


In [2]:
tf.random.set_seed(22)
np.random.seed(22)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [3]:
train_article_path = "sumdata/train/train.article.txt"
train_title_path = "sumdata/train/train.title.txt"
valid_article_path = "sumdata/train/valid.article.filter.txt"
valid_title_path = "sumdata/train/valid.title.filter.txt"

In [4]:
num_hidden = 1024
num_layers = 2
beam_size = 10
embedding_size = 128
lstm_size = 1024
learning_rate = 1e-3
batch_size = 10
buffer_size = 32
num_epochs = 20
keep_prob = 0.8

In [177]:
def process_text(text):
    # create a space between a word and the punctuation following it
    text = re.sub(r"([?.!,¿])", r" \1 ", text)
    text = re.sub(r'[" "]+', " ", text)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    text = re.sub(r"[^a-zA-Z?.!,¿]+", " ", text)
    text = text.strip()
    return text  

In [178]:
def load_data(path):
    text = []
    
    with open(path,encoding='utf-8') as f:
        for line in f:
            text_ = str(line.strip().split('\t'))
            text.append(process_text(text_))
    return text          

In [179]:
article = load_data(train_article_path)
print ('The length of article dataset is: {}'.format(len(article)))
article[:5]

The length of article dataset is: 100


['australia s current account deficit shrunk by a record . billion dollars lrb . billion us rrb in the june quarter due to soaring commodity prices , figures released monday showed .',
 'at least two people were killed in a suspected bomb attack on a passenger bus in the strife torn southern philippines on monday , the military said .',
 'australian shares closed down . percent monday following a weak lead from the united states and lower commodity prices , dealers said .',
 'south korea s nuclear envoy kim sook urged north korea monday to restart work to disable its nuclear plants and stop its typical brinkmanship in negotiations .',
 'south korea on monday announced sweeping tax reforms , including income and corporate tax cuts to boost growth by stimulating sluggish private consumption and business investment .']

In [180]:
title = load_data(train_title_path)
print ('The length of title dataset is: {}'.format(len(title)))
title[:5]

The length of title dataset is: 100


['australian current account deficit narrows sharply',
 'at least two dead in southern philippines blast',
 'australian stocks close down . percent',
 'envoy urges north korea to restart nuclear disablement',
 'skorea announces tax cuts to stimulate economy']

In [181]:
max_vocab_size = 2 ** 10
def build_vocabulary(texts,max_vocab_size):
    
    word_dict = dict()
    word_counts = collections.Counter(' '.join(texts).split(' ')).most_common()
    for word, _ in word_counts:
        word_dict[word] = len(word_dict)
                            
    reversed_dict = dict(zip(word_dict.values(), word_dict.keys()))
    
    return word_dict,reversed_dict
    
word_dict,reversed_dict = build_vocabulary(article + title,max_vocab_size)
vocab_size = len(word_dict)
print ('The cocabulary size of word dict is: {}'.format(vocab_size))
word_dict

The cocabulary size of word dict is: 1185


{'the': 0,
 '.': 1,
 ',': 2,
 'in': 3,
 'a': 4,
 'of': 5,
 'to': 6,
 'on': 7,
 'wednesday': 8,
 'and': 9,
 's': 10,
 'said': 11,
 'us': 12,
 'monday': 13,
 'for': 14,
 'as': 15,
 'its': 16,
 'by': 17,
 'prices': 18,
 'an': 19,
 'percent': 20,
 'against': 21,
 'at': 22,
 'it': 23,
 'down': 24,
 'new': 25,
 'but': 26,
 'from': 27,
 'with': 28,
 'close': 29,
 'over': 30,
 'shares': 31,
 'south': 32,
 'that': 33,
 'oil': 34,
 'has': 35,
 'is': 36,
 'after': 37,
 'president': 38,
 'dollars': 39,
 'southern': 40,
 'gulf': 41,
 'up': 42,
 'two': 43,
 'closed': 44,
 'dealers': 45,
 'korea': 46,
 'state': 47,
 'was': 48,
 'obama': 49,
 'lower': 50,
 'street': 51,
 'russia': 52,
 'not': 53,
 'georgia': 54,
 'were': 55,
 'wall': 56,
 'chief': 57,
 'lehman': 58,
 'hurricane': 59,
 'trade': 60,
 'raids': 61,
 'bolivia': 62,
 'ambassador': 63,
 'zimbabwe': 64,
 'deal': 65,
 'powerful': 66,
 'due': 67,
 'envoy': 68,
 'share': 69,
 'losses': 70,
 'which': 71,
 'amid': 72,
 'bank': 73,
 'police': 74,
 

In [182]:
!pip install nltk
from nltk.tokenize import word_tokenize

# 分词
def tokensize_text(text):
    words = []
    for w in word_tokenize(process_text(text)):
        words.append(w)
    return words

You are using pip version 9.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [183]:
artile_words = tokensize_text(article[0])
artile_words

['australia',
 's',
 'current',
 'account',
 'deficit',
 'shrunk',
 'by',
 'a',
 'record',
 '.',
 'billion',
 'dollars',
 'lrb',
 '.',
 'billion',
 'us',
 'rrb',
 'in',
 'the',
 'june',
 'quarter',
 'due',
 'to',
 'soaring',
 'commodity',
 'prices',
 ',',
 'figures',
 'released',
 'monday',
 'showed',
 '.']

In [184]:
def words2ids(text,text_dict):
    ids = []
    words = tokensize_text(text)
    for w in words:
        ids.append(text_dict[w])
        
    return ids

In [185]:
article_ids = words2ids(article[0],word_dict)
article_ids

[556,
 10,
 292,
 293,
 200,
 557,
 17,
 4,
 558,
 1,
 83,
 39,
 201,
 1,
 83,
 12,
 202,
 3,
 0,
 559,
 560,
 67,
 6,
 561,
 139,
 18,
 2,
 562,
 294,
 13,
 295,
 1]

In [186]:
def ids2words(ids,reversed_dict):
    words = []
    for id in ids:
        words.append(reversed_dict[id])
     
    return words

In [187]:
article_words = ids2words(article_ids,reversed_dict)
article_words

['australia',
 's',
 'current',
 'account',
 'deficit',
 'shrunk',
 'by',
 'a',
 'record',
 '.',
 'billion',
 'dollars',
 'lrb',
 '.',
 'billion',
 'us',
 'rrb',
 'in',
 'the',
 'june',
 'quarter',
 'due',
 'to',
 'soaring',
 'commodity',
 'prices',
 ',',
 'figures',
 'released',
 'monday',
 'showed',
 '.']

In [188]:
def build_data_int(text,word_dict):
    dataset = []
    for i in range(len(text)):
        line_indices = words2ids(text[i],word_dict)
        print (line_indices)
        dataset.append(line_indices)
    
    return dataset

In [192]:
article_train = build_data_int(article,word_dict)
title_train = build_data_int(title,word_dict)

[556, 10, 292, 293, 200, 557, 17, 4, 558, 1, 83, 39, 201, 1, 83, 12, 202, 3, 0, 559, 560, 67, 6, 561, 139, 18, 2, 562, 294, 13, 295, 1]
[22, 203, 43, 84, 55, 103, 3, 4, 296, 297, 204, 7, 4, 563, 564, 3, 0, 565, 566, 40, 205, 7, 13, 2, 0, 104, 11, 1]
[85, 31, 44, 24, 1, 20, 13, 105, 4, 298, 299, 27, 0, 140, 141, 9, 50, 139, 18, 2, 45, 11, 1]
[32, 46, 10, 206, 68, 567, 568, 207, 142, 46, 13, 6, 300, 569, 6, 570, 16, 206, 571, 9, 572, 16, 573, 574, 3, 143, 1]
[32, 46, 7, 13, 106, 575, 208, 209, 2, 576, 577, 9, 578, 208, 301, 6, 579, 302, 17, 580, 581, 582, 583, 9, 303, 107, 1]
[144, 69, 18, 44, 24, 1, 20, 13, 7, 56, 51, 210, 9, 304, 305, 306, 27, 307, 308, 108, 309, 310, 2, 45, 11, 1]
[85, 31, 44, 24, 1, 20, 13, 105, 4, 298, 299, 27, 0, 140, 141, 9, 50, 139, 18, 2, 45, 11, 1]
[211, 584, 311, 312, 2, 585, 313, 586, 587, 2, 106, 70, 5, 1, 83, 314, 14, 0, 109, 315, 5, 71, 23, 588, 7, 589, 590, 1]
[591, 145, 592, 316, 13, 593, 317, 146, 9, 318, 209, 33, 212, 11, 86, 594, 595, 596, 9, 34, 597,

[137, 135, 15, 66, 198, 291, 40, 100]
[12, 196, 7, 129, 26, 53, 67, 6, 101, 102, 555]
[521, 328, 42, 150, 6, 287, 1178, 522]
[52, 553, 23, 1179, 10, 523, 524, 3, 138, 156, 1180]
[40, 525, 172, 526, 72, 554, 5, 64, 1181]
[138, 102, 1182, 1183, 7, 529, 528]
[530, 10, 552, 1184, 531, 532]


In [193]:
# 添加 <start> 和 <end>
def encode(article,title):
    article_indices = [vocab_size] + words2ids(article,word_dict) + [vocab_size + 1]
    title_indices = [vocab_size] + words2ids(title,word_dict) + [vocab_size + 1]
    
    return article_indices, title_indices

In [194]:
article_indices, title_indices = encode(article[0],title[0])

print ('Article BOS index is :{}'.format(vocab_size))
print ('Article EOS index is :{}'.format(vocab_size + 1))

print ('Title BOS index is :{}'.format(vocab_size))
print ('Title EOS index is :{}'.format(vocab_size + 1))

print ((article[0]),(title[0]))
print ()
print (article_indices,title_indices)

Article BOS index is :1185
Article EOS index is :1186
Title BOS index is :1185
Title EOS index is :1186
australia s current account deficit shrunk by a record . billion dollars lrb . billion us rrb in the june quarter due to soaring commodity prices , figures released monday showed . australian current account deficit narrows sharply

[1185, 556, 10, 292, 293, 200, 557, 17, 4, 558, 1, 83, 39, 201, 1, 83, 12, 202, 3, 0, 559, 560, 67, 6, 561, 139, 18, 2, 562, 294, 13, 295, 1, 1186] [1185, 85, 292, 293, 200, 1083, 224, 1186]


In [195]:
def data_to_tensor(text):
    text_tensor = []
    for line in text:
        line_tensor = tf.convert_to_tensor(line)
        text_tensor.append(line_tensor)
    return text_tensor

article_tensor, title_tensor = data_to_tensor(article_train),data_to_tensor(title_train)

In [196]:
article_tensor[0]

<tf.Tensor: id=255899, shape=(32,), dtype=int32, numpy=
array([556,  10, 292, 293, 200, 557,  17,   4, 558,   1,  83,  39, 201,
         1,  83,  12, 202,   3,   0, 559, 560,  67,   6, 561, 139,  18,
         2, 562, 294,  13, 295,   1])>

In [197]:
title_tensor[0]

<tf.Tensor: id=255999, shape=(6,), dtype=int32, numpy=array([  85,  292,  293,  200, 1083,  224])>

In [198]:
max_length = 0
for i in range(len(article_tensor)):
    if len(article_tensor[i]) > max_length:
        max_length = len(article_tensor[i])
    else:
        continue

max_length

42

In [199]:
article_tensor = keras.preprocessing.sequence.pad_sequences(article_tensor,maxlen = max_length,padding = 'post')
title_tensor = keras.preprocessing.sequence.pad_sequences(title_tensor,maxlen = max_length,padding = 'post')

In [200]:
dataset = tf.data.Dataset.from_tensor_slices((article_tensor,title_tensor)).shuffle(buffer_size)
dataset = dataset.batch(batch_size,drop_remainder = True)
dataset

<BatchDataset shapes: ((10, 42), (10, 42)), types: (tf.int32, tf.int32)>

In [201]:
article_batch, title_batch = next(iter(dataset))

print (article_batch)
print ()
print (title_batch)

tf.Tensor(
[[ 19 645 221  35 646 647   9 112   3 342  28 648 649 650  21   4 113 651
  343 652   7   4 653  51   2  74   9 156 157  11  13   1   0   0   0   0
    0   0   0   0   0   0]
 [144  31  44  24   1  20  13   7  56  51 210   9 304 305 306  27 307 308
  108 309 310   2  45  11   1   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0]
 [ 22 203  43  84  55 103   3   4 296 297 204   7   4 563 564   3   0 565
  566  40 205   7  13   2   0 104  11   1   0   0   0   0   0   0   0   0
    0   0   0   0   0   0]
 [114  69  18  44 158 159  13  15  89 222   0 344   7  34  18  27  59 160
   15  23 345 147   0  12  41 161   2  45  11   1   0   0   0   0   0   0
    0   0   0   0   0   0]
 [  0 654 655 656  17  38 657 658   7 659 660   0 347 348 661   7  19  34
  349   5  39   4 350   9   4 302 154   5   1  20   2  23  48 106 351   1
    0   0   0   0   0   0]
 [361  69  18 227  24   1  20  13   2  15  89 680 681  37 166 120  10 682
    7   0 362   2  45  11   1   0   0   

In [202]:
article_batch.shape

TensorShape([10, 42])

In [203]:
from __future__ import absolute_import,print_function,division,unicode_literals
import tensorflow as tf 
import tensorflow.keras as keras
from tensorflow.keras import layers,losses,optimizers,metrics
import os

# ----------------------------
#  Encoder
#-----------------------------

class Encoder(keras.Model):

    def __init__(self,vocab_size,embedding_dim,encoder_units,batch_size):
        super(Encoder,self).__init__()

        self.batch_size = batch_size
        self.encoder_units = encoder_units
        self.embedding = layers.Embedding(vocab_size,embedding_dim)
        self.gru = layers.GRU(self.encoder_units,
                                    return_sequences = True,
                                    return_state = True,
                                    recurrent_initializer = 'glorot_uniform')

    def call(self,x,hidden):
        x = self.embedding(x)
        output,state = self.gru(x,initial_state = hidden)
        return output,state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_size,self.encoder_units))

# ----------------------------
#  Attention
#-----------------------------
class BahdanauAttention(keras.Model):

    def __init__(self,units):
        super(BahdanauAttention,self).__init__()

        self.W1 = layers.Dense(units)
        self.W2 = layers.Dense(units)
        self.V = layers.Dense(1)

    def call(self,query,values):
        # hidden shape == (batch_size,hidden size)
        # hidden_with_time_axis shape == (batch_size,1,hidden size)
        hidden_with_time_axis = tf.expand_dims(query,1)

        # score shape == (batch_size,max_length,1)
        score = self.V(tf.nn.tanh(
            self.W1(values) + self.W2(hidden_with_time_axis)
        ))

        # attention_weights shape == (batch_size,max_length,1)
        attention_weights = tf.nn.softmax(score,axis = 1)

        # context_vector shape after sum == (batch_size,hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector,axis = 1)

        return context_vector,attention_weights

# ----------------------------
#  Decoder
#-----------------------------
class Decoder(keras.Model):
    
    def __init__(self,vocab_size,embedding_dim,decoder_units,batch_size):
        super(Decoder,self).__init__()

        self.batch_size = batch_size
        self.decoder_units = decoder_units
        self.embedding = layers.Embedding(vocab_size,embedding_dim)
        self.gru = layers.GRU(self.decoder_units,
                                    return_sequences = True,
                                    return_state = True,
                                    recurrent_initializer = 'glorot_uniform')
        self.fc = layers.Dense(vocab_size)

        self.attention = BahdanauAttention(self.decoder_units)

    # encoder_output shape = (batch_size,max_length,hidden_size)
    def call(self,x,hidden,encoder_output):
        context_vector,attention_weights = self.attention(hidden,encoder_output)

        # x shape after passing through embedding == (batch_size,1,embedding_dim)
        x = self.embedding(x)

        # x shape after concatenation == (batch_size,1,embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector,1),x],axis = -1)

        # passing the concatenated vector to the GRU
        output,state = self.gru(x)

        # output shape == (batch_size * 1,hidden_size)
        output = tf.reshape(output,(-1,output.shape[2]))

        # output shape == (batch_size,vocab)
        x = self.fc(output)

        return x,state,attention_weights

In [204]:
vocab_size = vocab_size + 2
# Encoder
encoder = Encoder(vocab_size,embedding_size,lstm_size,batch_size)
sample_hidden = encoder.initialize_hidden_state()
sample_output,sample_hidden = encoder(article_batch,sample_hidden)
print ('Encoder output shape:(batch size,sequence length units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape:(batch_size,units) {}'.format(sample_hidden.shape))

# Attention
attention_layer = BahdanauAttention(10)
attention_result,attention_weights = attention_layer(sample_hidden,sample_output)
print ("Attention result shape:(batch size,units) {}".format(attention_result.shape))
print ("Attention weights shape:(batch size,sequence_length,1) {}".format(attention_weights.shape))

# Decoder
decoder = Decoder(vocab_size,embedding_size,lstm_size,batch_size)
sample_decoder_output,_,_ = decoder(tf.random.uniform((10,1)),sample_hidden,sample_output)
print ('Decoder output shape: (batch_size,vocab_size) {}'.format(sample_decoder_output.shape))

Encoder output shape:(batch size,sequence length units) (10, 42, 1024)
Encoder Hidden state shape:(batch_size,units) (10, 1024)
Attention result shape:(batch size,units) (10, 1024)
Attention weights shape:(batch size,sequence_length,1) (10, 42, 1)
Decoder output shape: (batch_size,vocab_size) (10, 1187)


In [207]:
optimizer = keras.optimizers.Adam()
loss_object = keras.losses.SparseCategoricalCrossentropy(from_logits=True,reduction='none')

#checkpoint_path = 'training_checkpoints'
#if not os.path.exists('training_checkpoints'):
#    os.mkdir('training_checkpoints')

#ckpt = tf.train.Checkpoint(optimizer = optimizer,
#                           encoder = encoder,
#                           decoder = decoder)
#
#ckpt_manager = tf.train.CheckpointManager(ckpt,checkpoint_path,max_to_keep=5)

#if ckpt_manager.latest_checkpoint:
#    ckpt.restore(ckpt_manager.latest_checkpoint)
#else:
#   print ('no saved checkpoint... ')

In [208]:
def loss_func(real,pred):
    mask = tf.math.logical_not(tf.math.equal(real,0))
    loss_ = loss_object(real,pred)
    
    mask = tf.cast(mask,dtype=loss_.dtype)
    loss_ *= mask
    
    return tf.reduce_mean(loss_)

In [209]:
@tf.function
def train_one_step(inp,targ,enc_hidden):
    batch_size,targ_length = targ.shape
    loss = 0
    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp,enc_hidden)
        dec_hidden = enc_hidden

        for t in range(targ_length - 1):
            dec_input = tf.expand_dims(targ[:,t],1) # using teacher forcing
            predictions,dec_hidden,_ = decoder(dec_input,dec_hidden,enc_output)
            loss += loss_func(targ[:,t + 1],predictions)

    batch_loss = loss / targ_length
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss,variables)
    optimizer.apply_gradients(zip(gradients,variables))

    return batch_loss

In [210]:
num_batch_per_epoch = (len(article_tensor) - 1) // batch_size  + 1
num_batch_per_epoch

10

In [ ]:
for epoch in range(num_epochs):
    start = time.time()
    
    enc_hidden = encoder.initialize_hidden_state()
    
    total_loss = 0
    
    for batch,(inp,targ) in enumerate(dataset.take(num_batch_per_epoch)):
        batch_loss = train_one_step(inp,targ,enc_hidden)
        total_loss += batch_loss
        
        if batch % 10 == 0:
            print ('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,batch,batch_loss.numpy()))
    if (epoch + 1) % 2 ==0:
        #ckpt_save_path = ckpt_manager.save()
        print ('Saving checkpoint for epoch {} at {}'.format(epoch + 1,ckpt_save_path))
        #
print ('Epoch {} loss {:.4f}'.format(epoch + 1,total_loss / num_batch_per_epoch))
print ('Time taken for one epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 1.0450
Epoch 2 Batch 0 Loss 0.9431
Saving checkpoint for epoch 2 at training_checkpoints\ckpt-22
Epoch 3 Batch 0 Loss 0.8618
Epoch 4 Batch 0 Loss 0.8344
Saving checkpoint for epoch 4 at training_checkpoints\ckpt-22
Epoch 5 Batch 0 Loss 0.8075


In [ ]:
text_sentence = 'the united nations  humanitarian chief john holmes arrived in ethiopia monday to tour regions affected by drought , which has left some eight million people in need of urgent food aid .'
raw_data = text_sentence
raw_data = process_text(raw_data)

raw_data_sequence= words2ids(raw_data,word_dict)
raw_data_sequence

In [ ]:
start_token = [vocab_size]
end_token = [vocab_size + 1]

raw_data_sequence = start_token + raw_data_sequence + end_token
raw_data_sequence
#raw_data_sequence = tf.convert_to_tensor(raw_data_sequence)
#raw_data_sequence

In [129]:
input_sequence = tf.expand_dims(raw_data_sequence,0)
input_sequence

<tf.Tensor: id=201072, shape=(1, 34), dtype=int32, numpy=
array([[1185,    0,  140,  319,  601,   57,  148,  602,  320,    3,  321,
          13,    6,  603,  322,  604,   17,  323,    2,   71,   35,  605,
         149,  606,  607,   84,    3,  608,    5,  609,  610,  324,    1,
        1186]])>

In [130]:
# Encoder
encoder = Encoder(vocab_size,embedding_size,lstm_size,batch_size)
en_initial_states = encoder.initialize_hidden_state()

en_outputs,en_state = encoder(input_sequence,en_initial_states)
en_outputs


InvalidArgumentError: indices[0,0] = 1185 is not in [0, 1185) [Op:ResourceGather] name: encoder_15/embedding_18/embedding_lookup/